In [ ]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
csv_path = '../../breast_density_calculation/breast_fgt_volumes.csv'
df = pd.read_csv(csv_path)
df

In [ ]:
df['dataset'] = df['filename'].str.split('_').str[0]
df['patient_name'] = df['filename'].str.split('.').str[0]
df

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

sns.boxplot(data=df, x='dataset', y='breast_density', ax=ax[0])
sns.boxplot(data=df, x='dataset', y='breast_volume_mm3', ax=ax[1])
sns.boxplot(data=df, x='dataset', y='parenchyma_volume_mm3', ax=ax[2])
plt.tight_layout()
plt.show()

In [ ]:
df.groupby('dataset')['breast_density'].quantile([0.25, 0.5, 0.75])

In [ ]:
df['breast_density'].quantile([0.25, 0.5, 0.75])

In [ ]:
def classify_breast_density(density: float) -> str:
    if density < 0.04287:
        return 'a'
    elif density < 0.07909:
        return 'b'
    elif density < 0.13591:
        return 'c'
    else:
        return 'd'
    
df['breast_density_class'] = df['breast_density'].map(classify_breast_density)

In [ ]:
sns.stripplot(data=df, x='dataset', y='breast_density', hue='breast_density_class')
plt.show()

In [ ]:
input_folder = '../../deep_learning/classification_data_version_3/labels'

for json_file in os.listdir(input_folder):
    print(f'Processing file {json_file}')
    file_path = os.path.join(input_folder, json_file)
    with open(file_path, 'r') as file:
        labels = json.load(file)
    if not labels['breast_density'] in ['a', 'b', 'c', 'd']:
        patient_name = json_file.split('.')[0]
        breast_density = df[df['patient_name'] == patient_name]['breast_density_class'].to_list()[0]
        labels['breast_density'] = breast_density
        with open(file_path, 'w') as file:
            json.dump(labels, file)
    